In [23]:
import os
import argparse
import pickle

import pandas as pd

from sklearn.model_selection import StratifiedKFold, train_test_split
from datasets import load_metric
#from evaluate import load

from seqeval.metrics import classification_report
import torch

# huggingface tokenizer/model
from transformers import AutoModelForTokenClassification

# huggingface trainer
from transformers import Trainer
from transformers import TrainingArguments


# Customize encoder
from ner.ner_dataset import NERCollator
from ner.ner_dataset import NERDatasetPreEncoded


python hf_trainer.py --model_folder models --data_fn data/encoded/train.klue_roberta-base.encoded.pickle --use_kfold --n_splits 5

# **batch size = 1, 2, 4, 8, ... 조절하기 (2^k)**

In [ ]:
batch_size = 4

In [ ]:
import os
import argparse

os.environ["WANDB_DISABLED"] = "true"

def define_argparser():
    p = argparse.ArgumentParser()

    p.add_argument('--model_folder', required=False, default = 'models', help="Directory to save trained model.")
    p.add_argument('--data_fn', required=False, default = 'data/encoded/train.klue_roberta-base.encoded.pickle', help="Data file name encoded by encoding.py to train the model.")

    p.add_argument('--valid_ratio', type=float, default= 0.2)
    p.add_argument('--batch_size_per_device', type=int, default=batch_size)
    p.add_argument('--n_epochs_per_fold', type=int, default=50)
    p.add_argument('--warmup_ratio', type=float, default= 0.2)
    p.add_argument('--max_length', type=int, default=512)
    #p.add_argument('--max_length', type=int, default=100)

    p.add_argument('--use_kfold', action='store_true', default= True)
    p.add_argument('--n_splits', type=int, default=5)
    p.add_argument('--fold_i', type=int, default=None, help="It would be used to train with a certain fold of data due to resource limitation.")

    config = p.parse_args(args=[])

    return config




In [25]:
config = define_argparser()
print(config)

Namespace(model_folder='models', data_fn='data/encoded/train.klue_roberta-base.encoded.pickle', valid_ratio=0.2, batch_size_per_device=4, n_epochs_per_fold=50, warmup_ratio=0.2, max_length=512, use_kfold=True, n_splits=5, fold_i=None)


In [26]:
def load_data(fn, use_kfold=False, n_splits=5, shuffle=True):
    """
    Load tsv data as Dataframe.
    If use_kfold is true, a new column ['fold'] will be added for indexing each fold.
    load_data라는 이름의 함수를 정의합니다. 이 함수는 네 개의 파라미터를 받습니다:
    fn: 데이터 파일의 이름 또는 경로입니다.
    use_kfold: k-fold 교차 검증을 사용할지 여부를 나타내는 부울 값입니다. 기본값은 False입니다.
    n_splits: 교차 검증에서 몇 개의 fold로 나눌지를 결정하는 정수입니다. 기본값은 5입니다.
    shuffle: 데이터를 셔플할지 여부를 나타내는 부울 값입니다. 기본값은 True입니다.
    """
    # Get sentences and labels from a dataframe.
    with open(fn, "rb") as f: # fn 경로의 파일을 바이너리 읽기 모드("rb")로 엽니다. f는 파일 객체를 참조하는 변수입니다.
        dataset = pickle.load(f) # pickle 모듈을 사용하여 파일에서 객체를 역직렬화합니다. 이를 통해 저장된 데이터가 dataset 변수에 로드됩니다.
    data = pd.DataFrame(dataset.pop('data')) # dataset 딕셔너리에서 'data' 키를 제거하면서 해당 값을 가져와 pandas 데이터프레임으로 변환합니다.

    if use_kfold:
        skf = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=shuffle)
        # n_splits, random_state=42, shuffle 값을 사용하여 StratifiedKFold 객체를 생성합니다. 이 객체는 클래스 비율을 유지하면서 데이터를 fold로 나눕니다.
        data['fold'] = -1 # 데이터프레임에 'fold'라는 새 열을 추가하고, 모든 값을 -1로 초기화합니다.
        for n_fold, (_, v_idx) in enumerate(skf.split(data, data['sentence_class'])):
            # StratifiedKFold의 split 메소드를 사용해 생성된 인덱스를 반복하면서 각 fold의 검증 인덱스를 가져옵니다.
            data.loc[v_idx, 'fold'] = n_fold
            # 검증 인덱스(v_idx)에 해당하는 데이터의 'fold' 열 값을 현재 fold 번호(n_fold)로 설정합니다.
        data['id'] = [x for x in range(len(data))]
        # 각 데이터 포인트에 대해 유일한 ID를 생성하여 'id' 열에 할당합니다.

    return data, dataset

In [27]:
with open(config.data_fn, "rb") as f:
        org_data = pickle.load(f)

#이전에 encoding.py에서 저장해 놓은 데이터
org_data.keys()

dict_keys(['data', 'label_info', 'pad_token', 'pretrained_model_name'])

In [28]:
#org_data에서 'data'열과 나머지 열을 분리함
data, data_args = load_data(config.data_fn, use_kfold=config.use_kfold,
                     n_splits=config.n_splits, shuffle=True)

/home/lsj/바탕화면/pytorch-ko-ner-v2_analysis_03/.conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


In [29]:
data

,input_ids,attention_mask,labels,sentence_class,fold,id
0,"[0, 544, 15229, 1545, 11610, 2446, 1545, 2302,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",MC,0,0
1,"[0, 6146, 1258, 2137, 2383, 2]","[1, 1, 1, 1, 1, 1]","[-100, 0, 9, 10, 10, -100]",NM,2,1
2,"[0, 15229, 1545, 9888, 2446, 16, 1545, 5281, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, ...",NC,2,2
3,"[0, 12, 21, 13, 3729, 2145, 555, 2073, 5183, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17,...",NM,2,3
4,"[0, 12, 22, 13, 9679, 2052, 3817, 7750, 19521,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 17, 0, 0, 0, 0, 0, 0, 0, 29, 3...",QT,4,4
...,...,...,...,...,...,...
265,"[0, 9964, 2079, 4121, 2]","[1, 1, 1, 1, 1]","[-100, 0, 0, 0, -100]",Out,0,265
266,"[0, 15229, 1545, 26462, 2446, 16, 1545, 19578,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Out,0,266
267,"[0, 21, 18, 5025, 2125, 15463, 2]","[1, 1, 1, 1, 1, 1, 1]","[-100, 0, 0, 19, 20, 20, -100]",DP,4,267
268,"[0, 21, 18, 4103, 2178, 2088, 12, 5331, 2612, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Out,3,268


In [30]:
data_args

{'label_info': {'label_list': ['CN',
   'NC',
   'RL',
   'LC',
   'NM',
   'GD',
   'AG',
   'JB',
   'PS',
   'DP',
   'TI',
   'MC',
   'PC',
   'BP',
   'QT',
   'RA',
   'EQ'],
  'label_to_index': {'O': 0,
   'B-CN': 1,
   'I-CN': 2,
   'B-NC': 3,
   'I-NC': 4,
   'B-RL': 5,
   'I-RL': 6,
   'B-LC': 7,
   'I-LC': 8,
   'B-NM': 9,
   'I-NM': 10,
   'B-GD': 11,
   'I-GD': 12,
   'B-AG': 13,
   'I-AG': 14,
   'B-JB': 15,
   'I-JB': 16,
   'B-PS': 17,
   'I-PS': 18,
   'B-DP': 19,
   'I-DP': 20,
   'B-TI': 21,
   'I-TI': 22,
   'B-MC': 23,
   'I-MC': 24,
   'B-PC': 25,
   'I-PC': 26,
   'B-BP': 27,
   'I-BP': 28,
   'B-QT': 29,
   'I-QT': 30,
   'B-RA': 31,
   'I-RA': 32,
   'B-EQ': 33,
   'I-EQ': 34},
  'index_to_label': {0: 'O',
   1: 'B-CN',
   2: 'I-CN',
   3: 'B-NC',
   4: 'I-NC',
   5: 'B-RL',
   6: 'I-RL',
   7: 'B-LC',
   8: 'I-LC',
   9: 'B-NM',
   10: 'I-NM',
   11: 'B-GD',
   12: 'I-GD',
   13: 'B-AG',
   14: 'I-AG',
   15: 'B-JB',
   16: 'I-JB',
   17: 'B-PS',
   18: 'I-PS

In [31]:
index_to_label = data_args['label_info']['index_to_label']


In [32]:
print(data['sentence_class'].value_counts())

sentence_class
Out    78
NM     34
DP     31
NC     22
JB     13
QT     12
PS     12
AG     11
TI      9
RL      9
LC      8
PC      8
CN      8
BP      7
MC      6
GD      2
Name: count, dtype: int64


In [33]:
def split_dataset(data, use_kfold=False, n_fold=None, valid_ratio=.2, shuffle=False):
    """
    Split data into train and validation.
    Size of validation set will be determined by 'n_fold' when 'use_kfold' is True, otherwise determined by 'valid_ratio'.
    'shuffle' will affect only in case of 'use_kfold' is False.
    """

    if use_kfold == True:
        train = data[data['fold'] != n_fold]
        # data 데이터프레임에서 'fold' 열의 값이 n_fold와 다른 모든 데이터를 학습 데이터로 선택합니다.
        valid = data[data['fold'] == n_fold]
        # data 데이터프레임에서 'fold' 열의 값이 n_fold와 같은 데이터를 검증 데이터로 선택합니다.
    else:
        train, valid = train_test_split(
            data, test_size=valid_ratio, random_state=42, shuffle=shuffle, stratify=data['sentence_class'])
            #data, test_size=valid_ratio, random_state=42, shuffle=shuffle)

    train_dataset = NERDatasetPreEncoded(train['input_ids'].values, train['attention_mask'].values, train['labels'].values)
    valid_dataset = NERDatasetPreEncoded(valid['input_ids'].values, valid['attention_mask'].values, valid['labels'].values)
    # train 데이터프레임에서 필요한 열('input_ids', 'attention_mask', 'labels')을 추출하여 NERDatasetPreEncoded 클래스의 인스턴스를 생성합니다.
    # 이 클래스는 NER(Named Entity Recognition) 작업을 위한 데이터셋을 준비합니다.
    return train_dataset, valid_dataset

In [34]:
n_fold=1

pretrained_model_name = data_args['pretrained_model_name'].replace('/', '_')
# data_args 딕셔너리에서 사전 훈련된 모델의 이름을 가져와서, 이름에 포함된 모든 '/' 문자를 '_'로 변경합니다.
# 파일 시스템에서 경로 구분자로 사용되는 '/'를 파일 이름으로 사용할 수 없기 때문입니다.

label_to_index = data_args['label_info']['label_to_index']
index_to_label = data_args['label_info']['index_to_label']
pad_token = data_args['pad_token']

train_dataset, valid_dataset = split_dataset(
    data, use_kfold=config.use_kfold, n_fold=n_fold, valid_ratio=config.valid_ratio, shuffle=True)
print(
    '|train| =', len(train_dataset),
    '|valid| =', len(valid_dataset),
)


|train| = 216 |valid| = 54


In [35]:
def get_pretrained_model(model_name: str, num_labels: int):
    """
    Basically, use AutoModelForTokenClassification from Huffingface.
    This function remains for future issue.
    """
    model_loader = AutoModelForTokenClassification
    return model_loader.from_pretrained(model_name, num_labels=num_labels)

In [36]:
model = get_pretrained_model(
        data_args['pretrained_model_name'], len(label_to_index))

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
total_batch_size = config.batch_size_per_device * torch.cuda.device_count()
    # 전체 배치 크기, 총 반복 횟수, 워밍업 단계 수를 계산합니다. 이는 GPU 개수와 설정을 통해 동적으로 결정됩니다.
n_total_iterations = int(len(train_dataset) /
                         total_batch_size * config.n_epochs_per_fold)
n_warmup_steps = int(n_total_iterations * config.warmup_ratio)
print(
    '# of total_iters =', n_total_iterations,
    '# of warmup_iters =', n_warmup_steps,
)


# of total_iters = 2700 # of warmup_iters = 540


In [38]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.46.3', '1.1.1')

In [39]:
training_args = TrainingArguments(
    output_dir=f"./checkpoints/{pretrained_model_name}.{n_fold}",
    num_train_epochs=config.n_epochs_per_fold,
    per_device_train_batch_size=config.batch_size_per_device,
    per_device_eval_batch_size=config.batch_size_per_device,
    warmup_steps=n_warmup_steps,
    weight_decay=0.01,
    fp16=True,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,  # 체크포인트 최대 1개로 제한

    logging_steps=max(1,n_total_iterations // 100),
    save_steps=n_total_iterations // config.n_epochs_per_fold,
    load_best_model_at_end=True,
)

/home/lsj/바탕화면/pytorch-ko-ner-v2_analysis_03/.conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [40]:
class compute_metrics():

    def __init__(self, index_to_label):
        self.index_to_label = index_to_label

    def __call__(self, pred):
        """
        Compute metrics use "seqeval"
        It evaluates based on Entity Level F1 score.
        """
        metric = load_metric('seqeval')

        labels = pred.label_ids
        predictions = pred.predictions.argmax(2)

        # Discard special tokens based on true_labels.
        true_predictions = [[self.index_to_label[p] for p, l in zip(
            prediction, label) if l >= 0] for prediction, label in zip(predictions, labels)]
        true_labels = [[self.index_to_label[l] for p, l in zip(prediction, label) if l >= 0]
                    for prediction, label in zip(predictions, labels)]

        results = metric.compute(
            predictions=true_predictions, references=true_labels)
        eval_results = {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }
        print(classification_report(true_labels, true_predictions))

        return eval_results

In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=NERCollator(pad_token=pad_token,
                              with_text=False),
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics(index_to_label),
)

In [42]:
trainer.train()

  1%|          | 28/2700 [00:05<09:47,  4.55it/s]

{'loss': 3.5329, 'grad_norm': 9.7932767868042, 'learning_rate': 2.3148148148148148e-06, 'epoch': 0.5}


  2%|▏         | 54/2700 [00:11<09:22,  4.71it/s]

{'loss': 2.5653, 'grad_norm': 9.613458633422852, 'learning_rate': 4.814814814814815e-06, 'epoch': 1.0}


/home/lsj/바탕화면/pytorch-ko-ner-v2_analysis_03/.conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lsj/바탕화면/pytorch-ko-ner-v2_analysis_03/.conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

  2%|▏         | 54/2700 [00:13<09:22,  4.71it/s]

...

              precision    recall  f1-score   support

          AG       1.00      1.00      1.00         2
          BP       0.75      0.92      0.83        13
          CN       1.00      1.00      1.00         2
          DP       0.75      1.00      0.86         6
          EQ       0.80      1.00      0.89         4
          GD       1.00      1.00      1.00         1
          JB       0.67      0.67      0.67         3
          LC       0.84      1.00      0.91        16
          MC       0.40      1.00      0.57         2
          NC       1.00      1.00      1.00        27
          NM       1.00      1.00      1.00        32
          PC       1.00      0.75      0.86         4
          PS       1.00      1.00      1.00        40
          QT       0.50      0.50      0.50         2
          RL       0.67      0.33      0.44         6
          TI       1.00      0.94      0.97        18

   micro avg       0.91      0.95      0.93       178
   macro avg       0.84   

100%|██████████| 2700/2700 [12:40<00:00,  5.34it/s]

{'train_runtime': 760.3166, 'train_samples_per_second': 14.205, 'train_steps_per_second': 3.551, 'train_loss': 0.12428200391293676, 'epoch': 50.0}


100%|██████████| 2700/2700 [12:40<00:00,  3.55it/s]


TrainOutput(global_step=2700, training_loss=0.12428200391293676, metrics={'train_runtime': 760.3166, 'train_samples_per_second': 14.205, 'train_steps_per_second': 3.551, 'total_flos': 358534639166760.0, 'train_loss': 0.12428200391293676, 'epoch': 50.0})

# **Result(batch 4, epoch 100, max_length 100)**


             precision    recall  f1-score   support

          AG       1.00      1.00      1.00         2
          BP       0.67      0.60      0.63        10
          CN       0.50      0.50      0.50         2
          DP       0.75      0.75      0.75         8
          EQ       1.00      0.60      0.75         5
          GD       1.00      1.00      1.00         2
          JB       0.86      1.00      0.92         6
          LC       0.81      0.89      0.85        19
          MC       1.00      1.00      1.00         1
          NC       0.95      0.90      0.93        21
          NM       0.93      1.00      0.97        28
          PC       0.33      0.50      0.40         2
          PS       1.00      1.00      1.00        35
          QT       0.67      0.80      0.73         5
          RL       0.75      0.60      0.67         5
          TI       0.80      0.84      0.82        19

   micro avg       0.87      0.88      0.87       170

   macro avg       0.81      0.81      0.81       170
   
weighted avg       0.87      0.88      0.87       170


{'eval_loss': 0.18080152571201324, 'eval_precision': 0.8670520231213873, 'eval_recall': 0.8823529411764706, 'eval_f1': 0.8746355685131196, 'eval_accuracy': 0.9728424864212432, 'eval_runtime': 1.3323, 'eval_samples_per_second': 40.531, 'eval_steps_per_second': 10.508, 'epoch': 100.0}


# **Result(batch 4, epoch 100, max_length 512)**

              precision    recall  f1-score   support

          BP       0.50      0.67      0.57         3
          CN       1.00      1.00      1.00         3
          DP       0.90      0.90      0.90        10
          EQ       0.00      0.00      0.00         0
          JB       0.00      0.00      0.00         2
          LC       0.58      0.70      0.64        20
          MC       0.00      0.00      0.00         1
          NC       0.88      0.88      0.88         8
          NM       0.98      1.00      0.99        49
          PC       0.50      1.00      0.67         1
          PS       0.93      0.98      0.96        66
          QT       0.50      0.50      0.50         4
          RL       0.33      0.17      0.22         6
          TI       0.72      0.68      0.70        19
          

   micro avg       0.84      0.86      0.85       192

   macro avg       0.56      0.61      0.57       192
   
weighted avg       0.83      0.86      0.85       192

# **Result(batch 1, epoch 100, max_length 512)**

              precision    recall  f1-score   support

          AG       1.00      1.00      1.00         3
          BP       0.78      0.70      0.74        20
          CN       1.00      1.00      1.00         1
          DP       0.60      0.86      0.71         7
          EQ       0.91      0.77      0.83        13
          GD       0.60      0.60      0.60         5
          JB       0.83      0.71      0.77         7
          LC       0.76      0.93      0.83        27
          MC       0.67      1.00      0.80         2
          NC       0.82      0.88      0.85        32
          NM       1.00      1.00      1.00        35
          PC       0.33      0.33      0.33         3
          PS       0.95      1.00      0.98        41
          QT       1.00      1.00      1.00         8
          RL       0.50      0.33      0.40         3
          TI       0.60      0.62      0.61        24

   micro avg       0.82      0.86      0.84       231

   macro avg       0.77      0.80      0.78       231
   
weighted avg       0.83      0.86      0.84       231

# **Result(batch 4, epoch 20, max_length 100)**

            precision    recall  f1-score   support

          AG       1.00      1.00      1.00         2
          BP       1.00      1.00      1.00         5
          CN       1.00      1.00      1.00         1
          DP       0.56      0.71      0.63         7
          EQ       1.00      1.00      1.00         3
          GD       0.60      0.75      0.67         4
          JB       0.50      0.67      0.57         3
          LC       0.70      0.82      0.76        17
          MC       1.00      1.00      1.00         1
          NC       0.91      1.00      0.95        20
          NM       0.94      0.97      0.95        30
          PC       1.00      1.00      1.00         1
          PS       1.00      1.00      1.00        36
          QT       0.80      1.00      0.89         4
          RL       1.00      0.33      0.50         3
          TI       0.85      0.85      0.85        13

   micro avg       0.87      0.92      0.89       150
   macro avg       0.87      0.88      0.86       150
weighted avg       0.88      0.92      0.89       150

{'eval_loss': 0.12187973409891129, 'eval_precision': 0.8679245283018868, 'eval_recall': 0.92, 'eval_f1': 0.8932038834951457, 'eval_accuracy': 0.9761215629522432, 'eval_runtime': 1.233, 'eval_samples_per_second': 43.795, 'eval_steps_per_second': 11.354, 'epoch': 20.0}

In [43]:
fn_prefix = '.'.join([pretrained_model_name, 
                    f"{config.n_epochs_per_fold}_epochs", 
                    f"{config.max_length}_length",
                    f"{n_fold}_fold", 
                    "pth"])
model_fn = os.path.join(config.model_folder, fn_prefix)

torch.save({
    'rnn': None,
    'cnn': None,
    'bert': trainer.model.state_dict(),
    'config': config,
    'vocab': None,
    'classes': index_to_label,
    'pretrained_model_name': data_args['pretrained_model_name']
}, model_fn)

In [44]:
model_fn

'models/klue_roberta-base.50_epochs.512_length.1_fold.pth'